IMPORTING PACKAGES


In [2]:
pip install catboost

     |████████████████████████████████| 65.7MB 56kB/s 


In [309]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
import pickle
from sklearn.model_selection import train_test_split, RandomizedSearchCV, StratifiedKFold, KFold, GridSearchCV,cross_val_score
from sklearn.metrics import f1_score, roc_auc_score,roc_curve,accuracy_score,confusion_matrix, precision_recall_curve, auc, roc_curve, recall_score, classification_report
import mpl_toolkits.mplot3d.axes3d as p3
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from sklearn.utils import resample,shuffle
from sklearn.decomposition import PCA,SparsePCA,KernelPCA,NMF
from holoviews import opts
from sklearn import metrics, mixture, cluster, datasets
from sklearn.mixture import GaussianMixture
from sklearn.calibration import CalibratedClassifierCV
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE
from catboost import CatBoostClassifier
from scipy.stats import randint
import lightgbm as lgb
import pandas_profiling as pp
import xgboost as xgb

In [310]:
from google.colab import files
uploaded = files.upload()


Saving train_Df64byy.csv to train_Df64byy (1).csv


In [311]:
from google.colab import files
uploaded = files.upload()


Saving test_YCcRUnU.csv to test_YCcRUnU (1).csv


In [312]:
train_df = pd.read_csv('train_Df64byy (1).csv')
test_df = pd.read_csv('test_YCcRUnU (1).csv')
#combine = pd.concat([train,test])

DATA ANALYSIS

In [313]:
train_df.head()

,ID,City_Code,Region_Code,Accomodation_Type,Reco_Insurance_Type,Upper_Age,Lower_Age,Is_Spouse,Health Indicator,Holding_Policy_Duration,Holding_Policy_Type,Reco_Policy_Cat,Reco_Policy_Premium,Response
0,1,C3,3213,Rented,Individual,36,36,No,X1,14+,3.0,22,11628.0,0
1,2,C5,1117,Owned,Joint,75,22,No,X2,NaN,NaN,22,30510.0,0
2,3,C5,3732,Owned,Individual,32,32,No,NaN,1.0,1.0,19,7450.0,1
3,4,C24,4378,Owned,Joint,52,48,No,X1,14+,3.0,19,17780.0,0
4,5,C8,2190,Rented,Individual,44,44,No,X2,3.0,1.0,16,10404.0,0


In [314]:
train_df.tail()

,ID,City_Code,Region_Code,Accomodation_Type,Reco_Insurance_Type,Upper_Age,Lower_Age,Is_Spouse,Health Indicator,Holding_Policy_Duration,Holding_Policy_Type,Reco_Policy_Cat,Reco_Policy_Premium,Response
50877,50878,C4,845,Rented,Individual,22,22,No,X3,NaN,NaN,18,7704.0,0
50878,50879,C5,4188,Rented,Individual,27,27,No,X3,7.0,3.0,4,5408.0,0
50879,50880,C1,442,Rented,Individual,63,63,No,X2,14+,1.0,12,11374.0,0
50880,50881,C1,4,Owned,Joint,71,49,No,X2,2.0,2.0,16,28179.2,1
50881,50882,C3,3866,Rented,Individual,24,24,No,X3,2.0,3.0,18,11424.0,0


In [315]:
train_df.describe()

,ID,Region_Code,Upper_Age,Lower_Age,Holding_Policy_Type,Reco_Policy_Cat,Reco_Policy_Premium,Response
count,50882.000000,50882.000000,50882.000000,50882.000000,30631.000000,50882.000000,50882.000000,50882.000000
mean,25441.500000,1732.788707,44.856275,42.738866,2.439228,15.115188,14183.950069,0.239947
std,14688.512535,1424.081652,17.310271,17.319375,1.025923,6.340663,6590.074873,0.427055
min,1.000000,1.000000,18.000000,16.000000,1.000000,1.000000,2280.000000,0.000000
25%,12721.250000,523.000000,28.000000,27.000000,1.000000,12.000000,9248.000000,0.000000
50%,25441.500000,1391.000000,44.000000,40.000000,3.000000,17.000000,13178.000000,0.000000
75%,38161.750000,2667.000000,59.000000,57.000000,3.000000,20.000000,18096.000000,0.000000
max,50882.000000,6194.000000,75.000000,75.000000,4.000000,22.000000,43350.400000,1.000000


In [316]:
 missed = train_df.isnull().sum()
 print(missed)

ID                             0
City_Code                      0
Region_Code                    0
Accomodation_Type              0
Reco_Insurance_Type            0
Upper_Age                      0
Lower_Age                      0
Is_Spouse                      0
Health Indicator           11691
Holding_Policy_Duration    20251
Holding_Policy_Type        20251
Reco_Policy_Cat                0
Reco_Policy_Premium            0
Response                       0
dtype: int64


In [317]:
train_df.shape

(50882, 14)

In [318]:
train_df.sort_values("ID",inplace=True,ascending=True)

In [319]:
test_df.sort_values("ID",inplace=True,ascending=True)

In [320]:
total = np.product(train_df.shape)
missing = missed.sum()
(missing/total)*100

7.326896404566307

In [321]:
#TREATING MISSING DATA
rev_null=["Health Indicator","Holding_Policy_Type","Holding_Policy_Duration"]


In [322]:
for column in rev_null:
    train_df[column].fillna(train_df[column].mode()[0], inplace=True)
    train_df[column].fillna(train_df[column].mode()[0], inplace=True)

In [323]:
for column in rev_null:
    test_df[column].fillna(test_df[column].mode()[0], inplace=True)
    test_df[column].fillna(test_df[column].mode()[0], inplace=True)

In [324]:
train_df['Health Indicator'] = pd.factorize(train_df["Health Indicator"])[0]
test_df['Health Indicator'] = pd.factorize(test_df["Health Indicator"])[0]

In [325]:
#fq = train_df.groupby(['Holding_Policy_Duration']).size()/len(train_df)    
# mapping values to dataframe 
#train_df.loc[:, "{}_freq_encode".format('Holding_Policy_Duration')] = train_df['Holding_Policy_Duration'].map(fq)   
# drop original column. 
#train_df = train_df.drop(['Holding_Policy_Duration'], axis = 1)

In [326]:
#fq = test_df.groupby(['Holding_Policy_Duration']).size()/len(test_df)    
# mapping values to dataframe 
#test_df.loc[:, "{}_freq_encode".format('Holding_Policy_Duration')] = test_df['Holding_Policy_Duration'].map(fq)   
# drop original column. 
#test_df = test_df.drop(['Holding_Policy_Duration'], axis = 1)

In [327]:
#fq = train_df.groupby(['Holding_Policy_Type']).size()/len(train_df)    
# mapping values to dataframe 
#train_df.loc[:, "{}_freq_encode".format('Holding_Policy_Type')] = train_df['Holding_Policy_Type'].map(fq)   
# drop original column. 
#train_df = train_df.drop(['Holding_Policy_Type'], axis = 1)

In [328]:
#fq = test_df.groupby(['Holding_Policy_Type']).size()/len(test_df)    
# mapping values to dataframe 
#test_df.loc[:, "{}_freq_encode".format('Holding_Policy_Type')] = test_df['Holding_Policy_Type'].map(fq)   
# drop original column. 
#test_df = test_df.drop(['Holding_Policy_Type'], axis = 1)

In [329]:
#fq = train_df.groupby(['Reco_Policy_Cat']).size()/len(train_df)    
# mapping values to dataframe 
#train_df.loc[:, "{}_freq_encode".format('Reco_Policy_Cat')] = train_df['Reco_Policy_Cat'].map(fq)   
# drop original column. 
#train_df = train_df.drop(['Reco_Policy_Cat'], axis = 1)

In [330]:
#fq = test_df.groupby(['Reco_Policy_Cat']).size()/len(test_df)    
# mapping values to dataframe 
#test_df.loc[:, "{}_freq_encode".format('Reco_Policy_Cat')] = test_df['Reco_Policy_Cat'].map(fq)   
# drop original column. 
#test_df = test_df.drop(['Reco_Policy_Cat'], axis = 1)

In [331]:
#fq = train_df.groupby(['Health Indicator']).size()/len(train_df)    
# mapping values to dataframe 
#train_df.loc[:, "{}_freq_encode".format('Health Indicator')] = train_df['Health Indicator'].map(fq)   
# drop original column. 
#train_df = train_df.drop(['Health Indicator'], axis = 1)

In [332]:
#fq = test_df.groupby(['Health Indicator']).size()/len(test_df)    
# mapping values to dataframe 
#test_df.loc[:, "{}_freq_encode".format('Health Indicator')] = test_df['Health Indicator'].map(fq)   
# drop original column. 
#test_df = test_df.drop(['Health Indicator'], axis = 1)

In [333]:
#fq = test_df.groupby(['City_Code']).size()/len(test_df)    
# mapping values to dataframe 
#test_df.loc[:, "{}_freq_encode".format('City_Code')] = test_df['City_Code'].map(fq)   
# drop original column. 
#test_df = test_df.drop(['City_Code'], axis = 1)

In [334]:
#fq = train_df.groupby(['City_Code']).size()/len(train_df)    
# mapping values to dataframe 
#train_df.loc[:, "{}_freq_encode".format('City_Code')] = train_df['City_Code'].map(fq)   
# drop original column. 
#train_df = train_df.drop(['City_Code'], axis = 1)

In [335]:
#train_df = train_df.dropna(subset=["Holding_Policy_Type","Holding_Policy_Duration"])

In [336]:
train_df.shape

(50882, 14)

In [337]:
test_df.shape

(21805, 13)

In [338]:
train_df.dtypes

ID                           int64
City_Code                   object
Region_Code                  int64
Accomodation_Type           object
Reco_Insurance_Type         object
Upper_Age                    int64
Lower_Age                    int64
Is_Spouse                   object
Health Indicator             int64
Holding_Policy_Duration     object
Holding_Policy_Type        float64
Reco_Policy_Cat              int64
Reco_Policy_Premium        float64
Response                     int64
dtype: object

In [339]:
Age = train_df['Upper_Age']+train_df['Lower_Age']


In [340]:
Age_1 = test_df['Upper_Age']+test_df['Lower_Age']

In [341]:
train_df.loc[:,'Age'] = (Age/2).astype(int)


In [342]:
test_df.loc[:,'Age'] = (Age_1/2).astype(int)

In [343]:
train_df.head()

,ID,City_Code,Region_Code,Accomodation_Type,Reco_Insurance_Type,Upper_Age,Lower_Age,Is_Spouse,Health Indicator,Holding_Policy_Duration,Holding_Policy_Type,Reco_Policy_Cat,Reco_Policy_Premium,Response,Age
0,1,C3,3213,Rented,Individual,36,36,No,0,14+,3.0,22,11628.0,0,36
1,2,C5,1117,Owned,Joint,75,22,No,1,1.0,3.0,22,30510.0,0,48
2,3,C5,3732,Owned,Individual,32,32,No,0,1.0,1.0,19,7450.0,1,32
3,4,C24,4378,Owned,Joint,52,48,No,0,14+,3.0,19,17780.0,0,50
4,5,C8,2190,Rented,Individual,44,44,No,1,3.0,1.0,16,10404.0,0,44


In [344]:
#fIND OUT ANY MISSING VALUES
train_df.isna().sum()

ID                         0
City_Code                  0
Region_Code                0
Accomodation_Type          0
Reco_Insurance_Type        0
Upper_Age                  0
Lower_Age                  0
Is_Spouse                  0
Health Indicator           0
Holding_Policy_Duration    0
Holding_Policy_Type        0
Reco_Policy_Cat            0
Reco_Policy_Premium        0
Response                   0
Age                        0
dtype: int64

DATA VISUALIZATION

In [345]:
sns.countplot(train_df.Response)

/usr/local/lib/python3.7/dist-packages/seaborn/_decorators.py:43: FutureWarning: Pass the following variable as a keyword arg: x. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  FutureWarning


In [346]:
train_df.Response.value_counts()

0    38673
1    12209
Name: Response, dtype: int64

In [347]:
sns.distplot(train_df.Age)

/usr/local/lib/python3.7/dist-packages/seaborn/distributions.py:2557: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)


In [348]:
sns.boxplot(y = 'Age', data = train_df,palette='Accent')

In [349]:
sns.scatterplot(x=train_df['Age'],y=train_df['Reco_Policy_Premium'])

In [350]:
sns.distplot(train_df.Reco_Policy_Premium)

/usr/local/lib/python3.7/dist-packages/seaborn/distributions.py:2557: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)


In [351]:
sns.boxplot(y = 'Reco_Policy_Premium', data = train_df,palette='Accent')

In [352]:
corr = train_df.corr()

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=bool))

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(10, 5))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(200, 20, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.5, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})


In [353]:
sns.scatterplot(x=train_df['Reco_Policy_Cat'],y=train_df['Reco_Policy_Premium'])

In [354]:
sns.barplot(x=train_df['Reco_Insurance_Type'],y=train_df['Response'])

In [355]:
sns.barplot(x=train_df['Is_Spouse'],y=train_df['Response'])

In [356]:
sns.barplot(x=train_df['Holding_Policy_Type'],y=train_df['Reco_Policy_Cat'])

In [357]:
sns.scatterplot(x=train_df['Holding_Policy_Duration'],y=train_df['Reco_Policy_Premium'])
sns.barplot(x=train_df['Holding_Policy_Duration'],y=train_df['Reco_Policy_Premium'])

In [358]:
sns.barplot(x=train_df['Holding_Policy_Type'],y=train_df['Holding_Policy_Duration'])

In [359]:
sns.catplot(x="Age", y="Reco_Policy_Cat",data=train_df, kind="bar");


DATA MANIPULATION

In [360]:
Accomodation_Type  = {'Owned': 1,'Rented': 0} 
train_df.Accomodation_Type = [Accomodation_Type[item] for item in train_df.Accomodation_Type]
test_df.Accomodation_Type = [Accomodation_Type[item] for item in test_df.Accomodation_Type]
print(train_df)

          ID City_Code  Region_Code  ...  Reco_Policy_Premium Response  Age
0          1        C3         3213  ...              11628.0        0   36
1          2        C5         1117  ...              30510.0        0   48
2          3        C5         3732  ...               7450.0        1   32
3          4       C24         4378  ...              17780.0        0   50
4          5        C8         2190  ...              10404.0        0   44
...      ...       ...          ...  ...                  ...      ...  ...
50877  50878        C4          845  ...               7704.0        0   22
50878  50879        C5         4188  ...               5408.0        0   27
50879  50880        C1          442  ...              11374.0        0   63
50880  50881        C1            4  ...              28179.2        1   60
50881  50882        C3         3866  ...              11424.0        0   24

[50882 rows x 15 columns]


In [361]:
Reco_Insurance_Type  = {'Joint': 1,'Individual': 0} 
train_df.Reco_Insurance_Type = [Reco_Insurance_Type[item] for item in train_df.Reco_Insurance_Type]
test_df.Reco_Insurance_Type = [Reco_Insurance_Type[item] for item in test_df.Reco_Insurance_Type]
print(train_df)

          ID City_Code  Region_Code  ...  Reco_Policy_Premium  Response  Age
0          1        C3         3213  ...              11628.0         0   36
1          2        C5         1117  ...              30510.0         0   48
2          3        C5         3732  ...               7450.0         1   32
3          4       C24         4378  ...              17780.0         0   50
4          5        C8         2190  ...              10404.0         0   44
...      ...       ...          ...  ...                  ...       ...  ...
50877  50878        C4          845  ...               7704.0         0   22
50878  50879        C5         4188  ...               5408.0         0   27
50879  50880        C1          442  ...              11374.0         0   63
50880  50881        C1            4  ...              28179.2         1   60
50881  50882        C3         3866  ...              11424.0         0   24

[50882 rows x 15 columns]


In [362]:
Is_Spouse = {'Yes':1,'No':0}
train_df.Is_Spouse = [Is_Spouse[item] for item in train_df.Is_Spouse]
test_df.Is_Spouse = [Is_Spouse[item] for item in test_df.Is_Spouse]
print(train_df)

          ID City_Code  Region_Code  ...  Reco_Policy_Premium  Response  Age
0          1        C3         3213  ...              11628.0         0   36
1          2        C5         1117  ...              30510.0         0   48
2          3        C5         3732  ...               7450.0         1   32
3          4       C24         4378  ...              17780.0         0   50
4          5        C8         2190  ...              10404.0         0   44
...      ...       ...          ...  ...                  ...       ...  ...
50877  50878        C4          845  ...               7704.0         0   22
50878  50879        C5         4188  ...               5408.0         0   27
50879  50880        C1          442  ...              11374.0         0   63
50880  50881        C1            4  ...              28179.2         1   60
50881  50882        C3         3866  ...              11424.0         0   24

[50882 rows x 15 columns]


In [363]:
train_df['City_Code'] = pd.factorize(train_df["City_Code"])[0]
train_df['Region_Code'] = pd.factorize(train_df["Region_Code"])[0]
train_df['Holding_Policy_Duration'] = pd.factorize(train_df["Holding_Policy_Duration"])[0]
test_df['City_Code'] = pd.factorize(test_df["City_Code"])[0]
test_df['Region_Code'] = pd.factorize(test_df["Region_Code"])[0]
test_df['Holding_Policy_Duration'] = pd.factorize(test_df["Holding_Policy_Duration"])[0]
train_df.head()

,ID,City_Code,Region_Code,Accomodation_Type,Reco_Insurance_Type,Upper_Age,Lower_Age,Is_Spouse,Health Indicator,Holding_Policy_Duration,Holding_Policy_Type,Reco_Policy_Cat,Reco_Policy_Premium,Response,Age
0,1,0,0,0,0,36,36,0,0,0,3.0,22,11628.0,0,36
1,2,1,1,1,1,75,22,0,1,1,3.0,22,30510.0,0,48
2,3,1,2,1,0,32,32,0,0,1,1.0,19,7450.0,1,32
3,4,2,3,1,1,52,48,0,0,0,3.0,19,17780.0,0,50
4,5,3,4,0,0,44,44,0,1,2,1.0,16,10404.0,0,44


In [364]:
train_df['Region_Code'] = pd.factorize(train_df["Region_Code"])[0]


In [365]:
test_df['Region_Code'] = pd.factorize(test_df["Region_Code"])[0]

In [366]:
train_df.head()

,ID,City_Code,Region_Code,Accomodation_Type,Reco_Insurance_Type,Upper_Age,Lower_Age,Is_Spouse,Health Indicator,Holding_Policy_Duration,Holding_Policy_Type,Reco_Policy_Cat,Reco_Policy_Premium,Response,Age
0,1,0,0,0,0,36,36,0,0,0,3.0,22,11628.0,0,36
1,2,1,1,1,1,75,22,0,1,1,3.0,22,30510.0,0,48
2,3,1,2,1,0,32,32,0,0,1,1.0,19,7450.0,1,32
3,4,2,3,1,1,52,48,0,0,0,3.0,19,17780.0,0,50
4,5,3,4,0,0,44,44,0,1,2,1.0,16,10404.0,0,44


In [367]:
test_df.head()

,ID,City_Code,Region_Code,Accomodation_Type,Reco_Insurance_Type,Upper_Age,Lower_Age,Is_Spouse,Health Indicator,Holding_Policy_Duration,Holding_Policy_Type,Reco_Policy_Cat,Reco_Policy_Premium,Age
0,50883,0,0,1,0,30,30,0,0,0,3.0,5,11934.0,30
1,50884,1,1,1,1,69,68,1,0,1,3.0,18,32204.8,68
2,50885,0,2,0,0,28,28,0,1,2,4.0,17,9240.0,28
3,50886,2,3,0,0,23,23,0,1,1,3.0,18,9086.0,23
4,50887,0,4,1,0,75,75,0,1,3,3.0,5,22534.0,75


In [368]:
train_df.isna().sum()

ID                         0
City_Code                  0
Region_Code                0
Accomodation_Type          0
Reco_Insurance_Type        0
Upper_Age                  0
Lower_Age                  0
Is_Spouse                  0
Health Indicator           0
Holding_Policy_Duration    0
Holding_Policy_Type        0
Reco_Policy_Cat            0
Reco_Policy_Premium        0
Response                   0
Age                        0
dtype: int64

In [369]:
train_df.dtypes

ID                           int64
City_Code                    int64
Region_Code                  int64
Accomodation_Type            int64
Reco_Insurance_Type          int64
Upper_Age                    int64
Lower_Age                    int64
Is_Spouse                    int64
Health Indicator             int64
Holding_Policy_Duration      int64
Holding_Policy_Type        float64
Reco_Policy_Cat              int64
Reco_Policy_Premium        float64
Response                     int64
Age                          int64
dtype: object

In [370]:
train_df = train_df.drop(['Region_Code'],axis = 1)
train_df = train_df.drop(['Upper_Age'],axis = 1)
test_df = test_df.drop(['Upper_Age'],axis = 1)
train_df = train_df.drop(['Lower_Age'],axis = 1)
test_df = test_df.drop(['Lower_Age'],axis = 1)
X = train_df.drop(['Response'],axis=1).values
y = train_df['Response'].values

In [371]:
test_df = test_df.drop(['Region_Code'],axis = 1)

In [372]:
train_df.head()

,ID,City_Code,Accomodation_Type,Reco_Insurance_Type,Is_Spouse,Health Indicator,Holding_Policy_Duration,Holding_Policy_Type,Reco_Policy_Cat,Reco_Policy_Premium,Response,Age
0,1,0,0,0,0,0,0,3.0,22,11628.0,0,36
1,2,1,1,1,0,1,1,3.0,22,30510.0,0,48
2,3,1,1,0,0,0,1,1.0,19,7450.0,1,32
3,4,2,1,1,0,0,0,3.0,19,17780.0,0,50
4,5,3,0,0,0,1,2,1.0,16,10404.0,0,44


PREDICTING THE MODEL

In [373]:
features = ["Health Indicator","Holding_Policy_Type","Reco_Policy_Cat","Holding_Policy_Duration","Accomodation_Type","Is_Spouse","City_Code","Age","Reco_Policy_Premium"]
target = "Response"
nums_cols = ["Age","Reco_Policy_Premium"]
features_1 = ["Health Indicator","Reco_Policy_Cat","Accomodation_Type","Is_Spouse","City_Code"]

In [374]:
train_target=train_df['Response']
train_df=train_df.drop(['Response'], axis = 1)
X_train,X_test,Y_train,Y_test = train_test_split(train_df,train_target,test_size =0.22,stratify = train_target, shuffle = True)

In [375]:
for column in features:
    X_train[column] = X_train[column].astype('int')
    X_test[column] = X_test[column].astype('int')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [376]:
sm = SMOTE(random_state=500)
X_res, y_res = sm.fit_resample(X_train, Y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [377]:
def training(model,X_train, Y_train):
    return model.fit(X_train, Y_train)

In [378]:
def evaluation_stats(model,X_train, X_test, Y_train, Y_test,algo,is_feature=False):
    print('Train Accuracy')
    y_pred_train = model.predict(X_train)                           
    print(accuracy_score(Y_train, y_pred_train))
    print('Validation Accuracy')
    y_pred_test = model.predict(X_test)                           
    print(accuracy_score(Y_test, y_pred_test))
    print("\n")
    print("Train AUC Score")
    print(roc_auc_score(Y_train, y_pred_train))
    print("Test AUC Score")
    print(roc_auc_score(Y_test, y_pred_test))
    
    if is_feature:
        plot_feature_importance(rf_model.feature_importances_,X.columns,algo)
def plot_feature_importance(importance,names,model_type):
    feature_importance = np.array(importance)
    feature_names = np.array(names)

In [379]:
##Using RANDOM FOREST METHOD
rf_model = training(RandomForestClassifier(),X_res, y_res)
evaluation_stats(rf_model,X_res, X_test, y_res, Y_test,'RANDOM FOREST')

Train Accuracy
1.0
Validation Accuracy
0.6585082626172398


Train AUC Score
1.0
Test AUC Score
0.5334436088738594


In [380]:
rf_model = training(RandomForestClassifier(criterion='entropy',n_estimators=5,max_depth=2),X_res, y_res)
evaluation_stats(rf_model,X_res, X_test, y_res, Y_test,'RANDOM FOREST')

Train Accuracy
0.6237402201299562
Validation Accuracy
0.5811523001339884


Train AUC Score
0.6237402201299562
Test AUC Score
0.5262507736760176


In [381]:
for column in features:
    X_train[column] = X_train[column].astype('int')
    X_test[column] = X_test[column].astype('int')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [382]:
#USING XGBOOST
space={ 'max_depth': hp.quniform("max_depth", 3,18,1),
        'gamma': hp.uniform ('gamma', 1,9),
        'reg_alpha' : hp.quniform('reg_alpha', 40,180,1),
        'reg_lambda' : hp.uniform('reg_lambda', 0,1),
        'colsample_bytree' : hp.uniform('colsample_bytree', 0.5,1),
        'min_child_weight' : hp.quniform('min_child_weight', 0, 10, 1),
        'n_estimators': 500,
        'seed': 0
      }

In [383]:
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe

In [384]:
def objective(space):

  clf=xgb.XGBClassifier(
                    n_estimators =space['n_estimators'], max_depth = int(space['max_depth']), gamma = space['gamma'],
                    reg_alpha = int(space['reg_alpha']),min_child_weight=int(space['min_child_weight']),
                    colsample_bytree=int(space['colsample_bytree']))
    
  evaluation = [( X_train, Y_train), ( X_test, Y_test)]
    
  clf.fit(X_train, Y_train,
            eval_set=evaluation, eval_metric="auc",
            early_stopping_rounds=10,verbose=False)
    

  pred = clf.predict(X_test)
  y_score = model.predict_proba(X_test)[:,1]
  accuracy = accuracy_score(Y_test, pred>0.5)
  Roc_Auc_Score = roc_auc_score(Y_test, y_score)
  print ("ROC-AUC Score:",Roc_Auc_Score)
  print ("SCORE:", accuracy)
  return {'loss': -Roc_Auc_Score, 'status': STATUS_OK }

In [385]:
trials = Trials()

In [386]:
random_search = {'criterion': ['entropy', 'gini'],
               'max_depth': [2,3,4,5,6,7,10],
               'min_samples_leaf': [4, 6, 8],
               'min_samples_split': [5, 7,10],
               'n_estimators': [300]}

clf = RandomForestClassifier()
model = RandomizedSearchCV(estimator = clf, param_distributions = random_search, n_iter = 10, 
                               cv = 4, verbose= 1, random_state= 101, n_jobs = -1)
model.fit(X_train,Y_train)

Fitting 4 folds for each of 10 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:  3.3min finished


RandomizedSearchCV(cv=4, error_score=nan,
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    max_samples=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
               

In [387]:
best_hyperparams = fmin(fn = objective,
                        space = space,
                        algo = tpe.suggest,
                        max_evals = 105,
                        trials = trials)

ROC-AUC Score:
0.6289891295511467
SCORE:
0.7600714604734257
ROC-AUC Score:
0.6289891295511467
SCORE:
0.7600714604734257
ROC-AUC Score:
0.6289891295511467
SCORE:
0.7600714604734257
ROC-AUC Score:
0.6289891295511467
SCORE:
0.7600714604734257
ROC-AUC Score:
0.6289891295511467
SCORE:
0.7600714604734257
ROC-AUC Score:
0.6289891295511467
SCORE:
0.7600714604734257
ROC-AUC Score:
0.6289891295511467
SCORE:
0.7600714604734257
ROC-AUC Score:
0.6289891295511467
SCORE:
0.7600714604734257
ROC-AUC Score:
0.6289891295511467
SCORE:
0.7600714604734257
ROC-AUC Score:
0.6289891295511467
SCORE:
0.7600714604734257
ROC-AUC Score:
0.6289891295511467
SCORE:
0.7600714604734257
ROC-AUC Score:
0.6289891295511467
SCORE:
0.7600714604734257
ROC-AUC Score:
0.6289891295511467
SCORE:
0.7600714604734257
ROC-AUC Score:
0.6289891295511467
SCORE:
0.7600714604734257
ROC-AUC Score:
0.6289891295511467
SCORE:
0.7600714604734257
ROC-AUC Score:
0.6289891295511467
SCORE:
0.7600714604734257
ROC-AUC Score:
0.6289891295511467
SCORE:

In [388]:
xgb_model=xgb.XGBClassifier(n_estimators = space['n_estimators'], max_depth = 10, gamma = 5.4567675434, reg_lambda = 0.555777,
                            reg_alpha = 8, min_child_weight=4.233432,colsample_bytree = 0.6)

In [389]:
xgb_model.fit(X_train,Y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.6, gamma=5.4567675434,
              learning_rate=0.1, max_delta_step=0, max_depth=10,
              min_child_weight=4.233432, missing=None, n_estimators=500,
              n_jobs=1, nthread=None, objective='binary:logistic',
              random_state=0, reg_alpha=8, reg_lambda=0.555777,
              scale_pos_weight=1, seed=None, silent=None, subsample=1,
              verbosity=1)

In [390]:
y_score = xgb_model.predict_proba(X_test)[:,1]
fpr, tpr, _ = roc_curve(Y_test, y_score)
plt.title('XGBoost ROC curve')
plt.xlabel('FPR (Precision)')
plt.ylabel('TPR (Recall)')
plt.plot(fpr,tpr)
plt.plot((0,1), ls='dashed',color='black')
plt.show()
print ('Area under curve (AUC): ', auc(fpr,tpr))

Area under curve (AUC):  0.6675239488441436


In [398]:
cat_pred = [pred[1] for pred in xgb_model.predict_proba(test_df)]
submissionC = pd.DataFrame(data = {'ID': test_df["ID"], 'Response': cat_pred})
submissionC.to_csv("final_solution.csv", index = False)
submissionC.head()


,ID,Response
0,50883,0.350011
1,50884,0.285130
2,50885,0.269361
3,50886,0.187171
4,50887,0.276041


LGBOOST

In [392]:
from lightgbm import LGBMClassifier

In [393]:
kfold, scores = KFold(n_splits = 5, shuffle = True, random_state = 25), list()
for train, test in kfold.split(X):
    X_train, X_test = X[train], X[test]
    Y_train, Y_test = y[train], y[test]
    LGB_model = LGBMClassifier(random_state=101, max_depth=80,n_estimators=10000,
                    reg_lambda=7.7073, reg_alpha=12, min_child_weight=4.233432,verbose= 1,
                    learning_rate = 0.05, gamma = 0.7, colsample_bytree = 0.9999,
                    eval_metric = 'auc', is_higher_better = 1, plot = True)
    LGB_model.fit(X_train, Y_train)

In [394]:
y_pred_lgbm_model = LGB_model.predict(X_test)
accuracy_score(Y_test, y_pred_lgbm_model)

0.7567806603773585

In [395]:
y_score = LGB_model.predict_proba(X_test)[:,1]
fpr, tpr, _ = roc_curve(Y_test, y_score)
plt.title('LGBoost ROC curve')
plt.xlabel('FPR (Precision)')
plt.ylabel('TPR (Recall)')
plt.plot(fpr,tpr)
plt.plot((0,1), ls='dashed',color='black')
plt.show()
print ('Area under curve (AUC): ', auc(fpr,tpr))

Area under curve (AUC):  0.6615222935550236


In [396]:
cat_pred = [pred[1] for pred in LGB_model.predict_proba(test_df)]
submissionC = pd.DataFrame(data = {'ID': test_df["ID"], 'Response': cat_pred})
submissionC.to_csv("v9.csv", index = False)
submissionC.head()


,ID,Response
0,50883,0.401266
1,50884,0.304277
2,50885,0.272932
3,50886,0.167382
4,50887,0.337419
